In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import nltk
import re
from bs4 import BeautifulSoup
import numpy as np
import torch
import transformers as ppb 

In [ ]:
import warnings
warnings.filterwarnings('ignore')

nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS =nltk.corpus.stopwords.words('english')

def clean_text(text):
    if type(text)!=str:
        return ""
    text = BeautifulSoup(text, "lxml").text # HTML decoding
    text = text.lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text
    text = BAD_SYMBOLS_RE.sub('', text) # delete symbols which are in BAD_SYMBOLS_RE from text
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # delete stopwors from text
    return text

In [ ]:
from transformers import OpenAIGPTTokenizer, OpenAIGPTModel
import torch
                                                                   #OpenAIGPTModel()
tokenizer = OpenAIGPTTokenizer.from_pretrained('openai-gpt')
model = OpenAIGPTModel.from_pretrained('openai-gpt')               #.OpenAIGPTConfig(n_positions=1207)
input_ids = torch.tensor(tokenizer.encode("Hello, my dog is cute", add_special_tokens=True)).unsqueeze(0)  # Batch size 1
#print(input_ids)
outputs = model(input_ids)
last_hidden_states = outputs[0]
#print(last_hidden_states)

ftfy or spacy is not installed using BERT BasicTokenizer instead of SpaCy & ftfy.


In [ ]:
def gpt_fscore(text,model,tokenizer):
    input_ids = torch.tensor(tokenizer.encode(text, add_special_tokens=True)).unsqueeze(0) 
    outputs = model(input_ids)
    last_hidden_states = outputs[0]
    return last_hidden_states

In [ ]:
fname = '/content/gdrive/My Drive/Gitter_Dataset/GitterCom.csv'
df = pd.read_csv(fname,encoding='utf8')
df.drop([581, 2102, 4858, 6417], axis=0, inplace=True)

non_string = []
for i, row in df.iterrows():
    res = ((type(row['message']) != str) or (type(row['Purpose']) != str) or (type(row['Category']) != str) or (type(row['Subcategory']) != str))
    if res:
        non_string.append(i)
df.drop(non_string, axis=0, inplace=True)
df['message'] = df['message'].apply(clean_text)

train_tokens = list(map(lambda t:   tokenizer.tokenize(t)+['[CLS]'], df['message']))
train_tokens_ids = list(map(tokenizer.convert_tokens_to_ids,train_tokens))
to_be_removed=[]

for i in range(0,len(train_tokens_ids)):
    if len(train_tokens_ids[i])>512:
        to_be_removed.append(i)
df.drop(to_be_removed,axis=0,inplace=True)
removed=0

for i in to_be_removed:
    train_tokens_ids.pop(i-removed)
    removed=removed+1
print(to_be_removed)

b=np.zeros((np.shape(df)[0]))
for i in range(0,np.shape(df)[0]):
    b[i]=np.shape(train_tokens_ids[i])[0]
pad=np.zeros((np.shape(df)[0],int(np.max(b))))

for i in range(0,np.shape(df)[0]):
    a=train_tokens_ids[i]
    for j in range(0,np.shape(a)[0]):
        pad[i,j]=a[j]

output = []

for i in range(20):
  pad1=pad[i*500:(i+1)*500,:]
  input_ids = torch.tensor(np.array(pad1))  
  with torch.no_grad():
    last_hidden_states = model(input_ids.long())    
  features = last_hidden_states[0][:,0,:].numpy()      
  output.append(features)     

len(output)

[]


20

In [ ]:
for i in range(1, 20):
  output[0] = np.vstack((output[0],output[i]))
output[0].shape

(9959, 768)

In [ ]:
fname = '8' + '.csv' 
np.savetxt(fname, output[0], delimiter=',', fmt='%f')